In [ ]:
import spacy
import numpy as np
#from pickle import dump,load
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import sequential, load_model
from keras.layers import Dense, LSTM , Embedding

In [ ]:
from google.colab import files 
files.upload()

Saving pg5200.txt to pg5200.txt


{'pg5200.txt': b'\xef\xbb\xbfThe Project Gutenberg EBook of Metamorphosis, by Franz Kafka\r\nTranslated by David Wyllie.\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **\r\n**     Please follow the copyright guidelines in this file.     **\r\n\r\n\r\nTitle: Metamorphosis\r\n\r\nAuthor: Franz Kafka\r\n\r\nTranslator: David Wyllie\r\n\r\nRelease Date: August 16, 2005 [EBook #5200]\r\nFirst posted: May 13, 2002\r\nLast updated: May 20, 2012\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK METAMORPHOSIS ***\r\n\r\n\r\n\r\n\r\nCopyright (C) 2002 David Wyllie.\r\n\r\n\r\n\r\n\r\n\r\n  Metamorphosis\r\n  Franz Kafka\r\n\r\nTranslated by David Wyllie\r\n\r\n\r\n\r\nI\r\n\r\n\r\nOn

In [ ]:
with open('pg5200.txt') as p:
  txt = p.read()
print("Text has this many characters : ", len(txt))

Text has this many characters :  139065


In [ ]:
NLP = spacy.load('en', disable = ['parser', 'tagger', 'ner'])
NLP.max_length = len(txt)*1.5
NLP

In [ ]:
tokens = [tok.text.lower() for tok in NLP(txt)  if tok.text not in '\n\n\n\n\n\n  \n!"-#$%&(?)-.*+,-/:;<=>@[\\]^_`{|}\t\n*** **** ']

In [ ]:
tokens[0]= 'the'

In [ ]:
len(tokens)

25564

In [ ]:
# l= t.split(" ")
    # for i in range(len(l)-3):
       #print(l[i:i+3])
      # print(l[i+3])


In [ ]:
x=[]
y=[]
for i in range(0,len(tokens) -25 -1):
  x.append(tokens[i:i+25])
  y.append(tokens[i+25: i+26])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

In [ ]:
x= tokenizer.texts_to_sequences(x)
y= tokenizer.texts_to_sequences(y)



In [ ]:
x=np.array(x)
y=np.array(y)
print(x.shape)
print(y.shape)

(25538, 25)
(25538, 1)


In [ ]:
y= to_categorical(y,num_classes = len(tokenizer.word_counts) + 1)

In [ ]:
y[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
for i in range(len(y[0])):
  if(y[0][i] == float(i)):
    print(i)

0


In [ ]:
len(y[0])

3041

In [ ]:
y.shape

(25538, 3041)

In [ ]:
from keras.models import Sequential
model= Sequential()
model.add(Embedding(len(tokenizer.word_counts) +1 , 25,input_length=25))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation = "relu"))
model.add(Dense(len(tokenizer.word_counts)+1,activation="softmax"))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics =['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            76025     
                                                                 
 lstm (LSTM)                 (None, 25, 50)            15200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 3041)              155091    
                                                                 
Total params: 269,066
Trainable params: 269,066
Non-trainable params: 0
_________________________________________________________________


In [ ]:
25538/250   #total batch to cover dataset

102.152

In [ ]:
model.fit(x,y,batch_size=250,epochs=350)

Epoch 1/350
103/103 [==============================] - 14s 101ms/step - loss: 6.7272 - accuracy: 0.0444
Epoch 2/350
103/103 [==============================] - 10s 99ms/step - loss: 6.1816 - accuracy: 0.0521
Epoch 3/350
103/103 [==============================] - 11s 104ms/step - loss: 6.1696 - accuracy: 0.0521
Epoch 4/350
103/103 [==============================] - 13s 130ms/step - loss: 6.1567 - accuracy: 0.0521
Epoch 5/350
103/103 [==============================] - 12s 120ms/step - loss: 6.0671 - accuracy: 0.0521
Epoch 6/350
103/103 [==============================] - 11s 102ms/step - loss: 5.9924 - accuracy: 0.0521
Epoch 7/350
103/103 [==============================] - 10s 99ms/step - loss: 5.8899 - accuracy: 0.0537
Epoch 8/350
103/103 [==============================] - 11s 107ms/step - loss: 5.7981 - accuracy: 0.0567
Epoch 9/350
103/103 [==============================] - 19s 187ms/step - loss: 5.7348 - accuracy: 0.0602
Epoch 10/350
103/103 [==============================] - 13s 128ms/

In [ ]:
dcti=list(tokenizer.word_index.keys())
for i in range(14,50):
  pred = np.reshape(x[i],(1,25))
  print(np.argmax(model.predict(x), axis=-1))

[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 169 ... 269  63  52]
[  3  15 1

In [ ]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'and': 3,
 'he': 4,
 'of': 5,
 'his': 6,
 'was': 7,
 'in': 8,
 'it': 9,
 'that': 10,
 'had': 11,
 'a': 12,
 'gregor': 13,
 'as': 14,
 'with': 15,
 'not': 16,
 'she': 17,
 "'s": 18,
 'for': 19,
 'would': 20,
 'him': 21,
 'her': 22,
 'at': 23,
 'but': 24,
 'on': 25,
 'they': 26,
 'all': 27,
 'this': 28,
 'be': 29,
 'from': 30,
 'you': 31,
 'room': 32,
 'could': 33,
 'if': 34,
 'or': 35,
 'out': 36,
 'have': 37,
 'there': 38,
 'father': 39,
 'been': 40,
 'sister': 41,
 'so': 42,
 'i': 43,
 'now': 44,
 'mother': 45,
 'project': 46,
 'gutenberg': 47,
 'himself': 48,
 'door': 49,
 'any': 50,
 'then': 51,
 'up': 52,
 'did': 53,
 'no': 54,
 'back': 55,
 'even': 56,
 'into': 57,
 'by': 58,
 'what': 59,
 'one': 60,
 'work': 61,
 'do': 62,
 'about': 63,
 'more': 64,
 'their': 65,
 'when': 66,
 'were': 67,
 'them': 68,
 'way': 69,
 'only': 70,
 "n't": 71,
 'time': 72,
 'other': 73,
 'is': 74,
 'than': 75,
 'tm': 76,
 'just': 77,
 'said': 78,
 'little': 79,
 'without': 80,
 'f

In [ ]:
model.save("95.h5")

NameError: ignored

In [ ]:
dump(tokenizer,open('tokenizer','wb'))

In [ ]:
t=txt[:100]

NameError: ignored